In [1]:
import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  
import calendar
import argparse
import glob

In [2]:
run = 'historical_run/'
dir_path = '/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_support_code/'+run




In [3]:
output_col_names = ['lon'
,'lat'
,'variable'
,'cohort_number'
,'stand_age'
,'potential_veg'
,'current_veg'
,'subtype'
,'community_type'
,'silt_clay'  ###check this order
,'lc_state'
,'land_area'
,'cohort_area'
,'year'
,'annual_sum'
,'monthly_maximum'
,'monthly_mean'
,'monthly_minimum'
,'Jan'
,'Feb'
,'Mar'
,'Apr'
,'May'
,'Jun'
,'Jul'
,'Aug'
,'Sep'
,'Oct'
,'Nov'
,'Dec'
,'region']

input_col_names = ["lon", 'lat','var' ,'Area', 'year', 'sum', 'max', 'average'
         , 'min', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'
         , 'Nov', 'Dec', 'Area_Name']

In [4]:
read_in_data = pd.read_csv('/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_support_code/'+run+'all_parallel_variable_info.csv')

In [5]:
read_in_data.loc[read_in_data['output_var'].isin(['SOILORGC', 'VEGC']) ].to_csv(dir_path+'VEGC_ORGC_info.csv', index=False)

In [6]:
read_in_data = pd.read_csv(dir_path+'VEGC_ORGC_info.csv')

In [8]:
read_in_data
read_in_data.loc[read_in_data['input_group'].isin([0
,23807
,20048 
,36337
,21301
,25060])]

,input_group,NIRR,PAR,PREC,TAIR,VPR,WS10,output_paths,output_group,output_var
0,0,clm_out/NIRR.csv0,clm_out/PAR.csv0,clm_out/PREC.csv0,clm_out/TAIR.csv0,clm_out/VPR.csv0,clm_out/WS10.csv0,var_out/SOILORGC.csv0,0,SOILORGC
1,0,clm_out/NIRR.csv0,clm_out/PAR.csv0,clm_out/PREC.csv0,clm_out/TAIR.csv0,clm_out/VPR.csv0,clm_out/WS10.csv0,var_out/VEGC.csv0,0,VEGC
20,20048,clm_out/NIRR.csv20048,clm_out/PAR.csv20048,clm_out/PREC.csv20048,clm_out/TAIR.csv20048,clm_out/VPR.csv20048,clm_out/WS10.csv20048,var_out/VEGC.csv20048,20048,VEGC
21,20048,clm_out/NIRR.csv20048,clm_out/PAR.csv20048,clm_out/PREC.csv20048,clm_out/TAIR.csv20048,clm_out/VPR.csv20048,clm_out/WS10.csv20048,var_out/SOILORGC.csv20048,20048,SOILORGC
22,21301,clm_out/NIRR.csv21301,clm_out/PAR.csv21301,clm_out/PREC.csv21301,clm_out/TAIR.csv21301,clm_out/VPR.csv21301,clm_out/WS10.csv21301,var_out/VEGC.csv21301,21301,VEGC
23,21301,clm_out/NIRR.csv21301,clm_out/PAR.csv21301,clm_out/PREC.csv21301,clm_out/TAIR.csv21301,clm_out/VPR.csv21301,clm_out/WS10.csv21301,var_out/SOILORGC.csv21301,21301,SOILORGC
26,23807,clm_out/NIRR.csv23807,clm_out/PAR.csv23807,clm_out/PREC.csv23807,clm_out/TAIR.csv23807,clm_out/VPR.csv23807,clm_out/WS10.csv23807,var_out/SOILORGC.csv23807,23807,SOILORGC
27,23807,clm_out/NIRR.csv23807,clm_out/PAR.csv23807,clm_out/PREC.csv23807,clm_out/TAIR.csv23807,clm_out/VPR.csv23807,clm_out/WS10.csv23807,var_out/VEGC.csv23807,23807,VEGC
30,25060,clm_out/NIRR.csv25060,clm_out/PAR.csv25060,clm_out/PREC.csv25060,clm_out/TAIR.csv25060,clm_out/VPR.csv25060,clm_out/WS10.csv25060,var_out/VEGC.csv25060,25060,VEGC
31,25060,clm_out/NIRR.csv25060,clm_out/PAR.csv25060,clm_out/PREC.csv25060,clm_out/TAIR.csv25060,clm_out/VPR.csv25060,clm_out/WS10.csv25060,var_out/SOILORGC.csv25060,25060,SOILORGC


In [4]:
i = 4 

In [ ]:
###read in cohort output
cohort_output = pd.read_csv(dir_path + read_in_data.loc[i, 'output_paths']
            , names = output_col_names)

print('cohort rows read in : '+ str(len(cohort_output)))

In [ ]:
cohort_output.subtype.unique()

In [ ]:
cohort_output.loc[cohort_output['subtype'] == 10]

In [33]:
forest_subtypes_num = [4, 5, 6, 10, 12]
forest_subtypes_names = ["Boreal", "Temperate Coniferous", "Temperate Deciduous", "Tropical", "Temperate Broadleaved Evergreen Forests"]

forest_subtypes = pd.DataFrame({
'subtype':forest_subtypes_num, 
    'forest_type':forest_subtypes_names
})


In [34]:
cohort_output=cohort_output.loc[(cohort_output['subtype'].isin(forest_subtypes['subtype']))& (cohort_output['year']>=1800)]

# cohort_output.loc[(cohort_output['subtype'].isin(forest_subtypes['subtype']))]
# len(cohort_output)

In [35]:
#### match pfts to the current veg, bin by standage for forests
cohort_output=cohort_output.merge(forest_subtypes, on = 'subtype', how = 'left')

intervals_standage = [-1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 3000]
cohort_output['stand_age_bin'] = pd.cut(
    cohort_output['stand_age'], bins=intervals_standage)
cohort_output['stand_age_interval_min'] = cohort_output['stand_age_bin'].apply(lambda x: x.left).astype(int) + 1

In [36]:
cohort_output

,lon,lat,variable,cohort_number,stand_age,potential_veg,current_veg,subtype,community_type,silt_clay,...,Jul,Aug,Sep,Oct,Nov,Dec,region,forest_type,stand_age_bin,stand_age_interval_min
0,-84.5,29.5,VEGC,2,3000,8,8,10,6,53.29,...,25429.04,25506.15,25620.77,25667.16,25645.93,25605.67,United_States_of_America,Tropical,"(150, 3000]",151
1,-84.5,29.5,VEGC,2,3000,8,8,10,6,53.29,...,25371.60,25434.75,25546.05,25633.62,25638.80,25613.44,United_States_of_America,Tropical,"(150, 3000]",151
2,-84.5,29.5,VEGC,2,3000,8,8,10,6,53.29,...,25433.73,25583.38,25703.20,25769.54,25760.45,25527.05,United_States_of_America,Tropical,"(150, 3000]",151
3,-84.5,29.5,VEGC,2,3000,8,8,10,6,53.29,...,25807.03,25917.74,26009.92,25999.65,25920.99,25843.15,United_States_of_America,Tropical,"(150, 3000]",151
4,-84.5,29.5,VEGC,2,3000,8,8,10,6,53.29,...,25649.45,25745.61,25823.97,25858.58,25848.24,25803.47,United_States_of_America,Tropical,"(150, 3000]",151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4071294,-78.0,55.0,VEGC,76,4,6,6,6,4,35.05,...,297.38,331.42,347.08,345.75,337.40,330.76,Canada,Temperate Deciduous,"(-1, 10]",0
4071295,-78.0,55.0,VEGC,77,3,6,6,6,4,35.05,...,107.87,125.40,133.88,134.60,130.74,127.47,Canada,Temperate Deciduous,"(-1, 10]",0
4071296,-78.0,55.0,VEGC,78,2,6,6,6,4,35.05,...,6.82,7.37,7.47,7.14,6.67,6.31,Canada,Temperate Deciduous,"(-1, 10]",0
4071297,-78.0,55.0,VEGC,79,1,6,6,6,4,35.05,...,6.98,8.79,9.75,9.89,9.52,9.19,Canada,Temperate Deciduous,"(-1, 10]",0


In [38]:
cohort_output['temp_weight'] = cohort_output['monthly_mean'] * cohort_output['cohort_area'] 


aggregations = {
 'cohort_area':'sum',
 'land_area':'sum',
    'temp_weight':'sum'
}


cohort_output = cohort_output.groupby(
    ['lon','lat','variable','forest_type','subtype','year','silt_clay','stand_age_interval_min']
)[[ 'cohort_area','land_area','temp_weight' ]].sum()


cohort_output['value_weight'] = cohort_output['temp_weight']/cohort_output['cohort_area']
cohort_output = cohort_output.reset_index()
print('cohort rows post cohort collapse : '+ str(len(cohort_output)))
# cohort_output

cohort rows post cohort collapse : 379688


In [42]:
cohort_output=cohort_output.drop(['temp_weight'], axis=1)

,lon,lat,variable,forest_type,subtype,year,silt_clay,stand_age_interval_min,cohort_area,land_area,value_weight
0,-84.5,29.5,VEGC,Tropical,10,1800,53.29,151,148,297,25513.140000
1,-84.5,29.5,VEGC,Tropical,10,1801,53.29,151,148,297,25476.770000
2,-84.5,29.5,VEGC,Tropical,10,1802,53.29,151,148,297,25525.300000
3,-84.5,29.5,VEGC,Tropical,10,1803,53.29,151,148,297,25686.370000
4,-84.5,29.5,VEGC,Tropical,10,1804,53.29,151,148,297,25687.280000
...,...,...,...,...,...,...,...,...,...,...,...
379683,-78.0,55.0,VEGC,Temperate Deciduous,6,2014,35.05,31,44,12830,3054.619318
379684,-78.0,55.0,VEGC,Temperate Deciduous,6,2014,35.05,41,36,12830,5978.675833
379685,-78.0,55.0,VEGC,Temperate Deciduous,6,2014,35.05,51,29,12830,5332.851379
379686,-78.0,55.0,VEGC,Temperate Deciduous,6,2014,35.05,61,8,5132,5781.522500


In [40]:
# lon_lat_pfts

cohort_output.to_csv('/home/smmrrr/TEM_output_processed/historical_run_model_data/'+read_in_data.loc[i, 'output_var']+str(read_in_data.loc[i, 'output_group'])+'.csv'
                        ,index = False)



